In [ ]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import os
import gc
import importlib
import multiprocessing

import sentiment_utils
importlib.reload(sentiment_utils)
from sentiment_utils import SentimentAnalysisModel, CommentsDataset, process_all_batches

In [ ]:
multiprocessing.set_start_method("fork", force=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
output_file_path = '../data/beer_reviews_text.csv'

num_rows= 6000

df = pd.read_csv(output_file_path, nrows=num_rows)
df.head()

In [ ]:
sentiment_model = SentimentAnalysisModel(model_path="../models/sentiment_model_1")

batch_size = 16
dataset = CommentsDataset(df["text"].astype(str).tolist())
data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=12,
    pin_memory=True,
    persistent_workers=True
)

df["sentiment_bert"] = process_all_batches(data_loader, sentiment_model)
df.head()


In [ ]:
torch.mps.empty_cache()
gc.collect()

In [ ]:
df